# Controlo da rota de 3 navios num lago infinito

In [2]:
from z3 import *
import numpy as np
from math import cos, sin

In [ ]:
VEL_INIT, VEL_HIGH, VEL_LOW = 0, 10, 1 # velocidade em m/s
THETA = 15 # angulo de viragem em graus
TAU = 0.01 # intervalo de tempo entre estados temporais consecutivos em s

GAUSS_DP = 80 # desvio padrao da gaussiana da inicializacao da posicao em m

In [ ]:
def declare_boat(boat_id):
    """
    A velocidade do state determina qual o seu modo:
        - INIT: vel = VEL_INIT
        - SAFE: vel = VEL_HIGH
        - IMMINENT COLLISION: vel = VEL_LOW
    """
    
    state = {
        "vel": Int(f"b{boat_id}_v"),
        "pos": {"x": Real(f"b{boat_id}_x"), "y": Real(f"b{boat_id}_y")},
        "ang": Int(f"b{boat_id}_a"),
        "time": Real(f"b{boat_id}_t")
    }
    
    return state


def init_boat(state):
    # cond_a = Or([state["ang"] == i*THETA for i in range(360/15)])
    
    # Randomly defined angle
    random_angle = np.random.shuffle([i*THETA for i in range(360/15)])[0]
    cond_a = state["ang"] == random_angle
    
    # Randomly defined position according to gaussian
    cond_x = state["pos"]["x"] == GAUSS_DP * np.random.randn()
    cond_y = state["pos"]["y"] == GAUSS_DP * np.random.randn()
    
    # Deterministic initial definitions
    cond_v = state["vel"] == VEL_INIT
    cond_t = state["time"] == 0
    
    return And(cond_a, cond_v, cond_t)


def untimed_to_imminent(state):
    return state["vel"] == VEL_LOW


def untimed_to_safe(state):
    return state["vel"] == VEL_HIGH


def timed_safe(state_ant, state_atual):
    # Definir evolucao do tempo
    cond_t = state_atual["time"] == state_ant["time"] + TAU
    
    # Definir evolucao da abcissa
    dx = state_ant["vel"] * cos(state_ant["ang"]) * TAU
    cond_x = state_atual["pos"]["x"] == state_ant["pos"]["x"] + dx
    
    # Definir evolucao da ordenada
    dy = state_ant["vel"] * sin(state_ant["ang"]) * TAU
    cond_y = state_atual["pos"]["y"] == state_ant["pos"]["y"] + dy
    
    return And(cond_t, cond_x, cond_y)


def timed_imminent(state_ant, state_atual):
    # Definir evolucao do tempo
    cond_t = state_atual["time"] == state_ant["time"] + TAU
    
    # Definir evolucao da abcissa
    dx = state_ant["vel"] * cos(state_ant["ang"]) * TAU
    cond_x = state_atual["pos"]["x"] == state_ant["pos"]["x"] + dx
    
    # Definir evolucao da ordenada
    dy = state_ant["vel"] * sin(state_ant["ang"]) * TAU
    cond_y = state_atual["pos"]["y"] == state_ant["pos"]["y"] + dy
    
    # Definir evolucao da rota
    theta_pos = state_atual["ang"] == state_ant["ang"] + THETA
    theta_neg = state_atual["ang"] == state_ant["ang"] - THETA
    cond_a = Or(theta_pos, theta_neg)
    
    return And(cond_t, cond_x, cond_y, cond_a)


def imminent_collision():
    